In [3]:
import datetime

print(datetime.datetime.now())

2022-09-03 14:58:23.641908


In [14]:
pd.read_csv('atendimentos.csv')

,ID,Tipo_Senha,Numeracao,Data_Emissao,Guiche,Data_Atendimento
0,1,SP,1,2022-09-03 14:58:23.641908,1.0,2022-09-03 15:08:23.641908
1,2,SE,1,2022-09-03 14:58:23.641909,1.0,2022-09-03 15:08:23.641909
2,3,SG,1,2022-09-03 14:58:23.641910,2.0,2022-09-03 15:08:23.641910
3,4,SP,2,2022-09-03 14:58:23.641911,1.0,2022-09-03 15:08:23.641911
4,5,SE,2,2022-09-03 14:58:23.641912,2.0,2022-09-03 15:08:23.641912
5,6,SP,3,2022-09-03 14:58:23.641913,2.0,2022-09-03 15:08:23.641913
6,7,SG,2,2022-09-03 14:58:23.641914,3.0,2022-09-03 15:08:23.641914
7,8,SG,3,2022-09-03 14:58:23.641915,NaN,NaN
8,9,SP,4,2022-09-03 14:58:23.641916,NaN,NaN
9,10,SG,4,2022-09-03 14:58:23.641917,NaN,NaN


In [12]:
import pandas as pd
tipo = "SP"
df = pd.read_csv('atendimentos.csv')
last_password = int(df[df["Tipo_Senha"] == tipo]["Numeracao"].max())
last_password

4

In [13]:
str(last_password).zfill(3)

'004'